In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

## WebScraping data from Oficial Neighbourhoods from RIO 

In [2]:
url_bairros = 'https://pt.wikipedia.org/wiki/Lista_de_bairros_da_cidade_do_Rio_de_Janeiro'

In [3]:
source = requests.get(url_bairros).text
soup = BeautifulSoup(source,'lxml')

In [4]:
# Find table from wiki
table = soup.find('table', class_='wikitable')
trs = table.find_all('tr')

In [5]:
# TABLE to LIST OF LISTS
bairros = []
for tr in trs:
    for td in tr.find_all('td'):
        row = []
        for a in td.find_all('a'):
            if a.text in ['Central','Sul','Oeste','Norte']:
                zona = a.text
            elif a.text in ['Bairro Imperial de São Cristóvão','Zona Sul','Barra da Tijuca e Baixada de Jacarepaguá','Grande Bangu','Zona Oeste','Grande Tijuca','Grande Méier','Ilha do Governador','Zona Norte']:
                sub = a.text
            elif a.text != '' and a.text[0] != '[' and a.text != 'Zona da Leopoldina':
                row = [zona,sub,a.text]
                bairros.append(row)
  

### Transform data into Dataframe

In [6]:
# LIST OF LISTS TO DATAFRAME
df = pd.DataFrame(bairros)
df.shape

(160, 3)

In [7]:
# Rename columns Names
df = df.rename(columns={0:'ZONA',1:'SUB',2:'BAIRRO'})
df.head()

,ZONA,SUB,BAIRRO
0,Central,Bairro Imperial de São Cristóvão,Benfica
1,Central,Bairro Imperial de São Cristóvão,Caju
2,Central,Bairro Imperial de São Cristóvão,Catumbi
3,Central,Bairro Imperial de São Cristóvão,Centro
4,Central,Bairro Imperial de São Cristóvão,Cidade Nova


In [8]:
# Error Centro Histórico e Zona Portuária
df.SUB.replace('Bairro Imperial de São Cristóvão','Centro Histórico e Zona Portuária', inplace = True)
imperial = {
    'ZONA' : 'Central',
    'SUB' : 'Centro Histórico e Zona Portuária',
    'BAIRRO' : 'São Cristóvão'
}
imperial = pd.DataFrame([imperial])
imperial = imperial.append(df , sort=True)

In [9]:
df = imperial
df = df.reset_index()
df = df.drop(['index'],axis=1)

In [10]:
df.head()

,BAIRRO,SUB,ZONA
0,São Cristóvão,Centro Histórico e Zona Portuária,Central
1,Benfica,Centro Histórico e Zona Portuária,Central
2,Caju,Centro Histórico e Zona Portuária,Central
3,Catumbi,Centro Histórico e Zona Portuária,Central
4,Centro,Centro Histórico e Zona Portuária,Central


**UPPER CASE**

In [11]:
df['ZONA'] = df['ZONA'].str.upper()
df['SUB'] = df['SUB'].str.upper()
df['BAIRRO'] = df['BAIRRO'].str.upper()

In [12]:
df['BAIRRO'] = df['BAIRRO'].str.replace('Á','A')
df['BAIRRO'] = df['BAIRRO'].str.replace('É','E')
df['BAIRRO'] = df['BAIRRO'].str.replace('Í','I')
df['BAIRRO'] = df['BAIRRO'].str.replace('Ó','O')
df['BAIRRO'] = df['BAIRRO'].str.replace('Ú','U')
df['BAIRRO'] = df['BAIRRO'].str.replace('Â','A')
df['BAIRRO'] = df['BAIRRO'].str.replace('Ê','E')
df['BAIRRO'] = df['BAIRRO'].str.replace('Ã','A')
df['BAIRRO'] = df['BAIRRO'].str.replace('Ç','C')

In [13]:
df.head()

,BAIRRO,SUB,ZONA
0,SAO CRISTOVAO,CENTRO HISTÓRICO E ZONA PORTUÁRIA,CENTRAL
1,BENFICA,CENTRO HISTÓRICO E ZONA PORTUÁRIA,CENTRAL
2,CAJU,CENTRO HISTÓRICO E ZONA PORTUÁRIA,CENTRAL
3,CATUMBI,CENTRO HISTÓRICO E ZONA PORTUÁRIA,CENTRAL
4,CENTRO,CENTRO HISTÓRICO E ZONA PORTUÁRIA,CENTRAL


# Get Latitude and Longitude

In [14]:
import geocoder

In [15]:
lat = []
long = []

for i in df['BAIRRO']:
    lat_lng_coords = None
    print(i)

    while(lat_lng_coords is None):
        g = geocoder.arcgis('{} , RIO DE JANEIRO'.format(i))
        lat_lng_coords = g.latlng
        print(g)
        
    lat.append(lat_lng_coords[0])
    long.append(lat_lng_coords[1])

SAO CRISTOVAO
<[OK] Arcgis - Geocode [São Cristóvão, Rio de Janeiro]>
BENFICA
<[OK] Arcgis - Geocode [Benfica, Rio de Janeiro]>
CAJU
<[OK] Arcgis - Geocode [Caju, Rio de Janeiro]>
CATUMBI
<[OK] Arcgis - Geocode [Catumbi, Rio de Janeiro]>
CENTRO
<[OK] Arcgis - Geocode [Centro, Bom Jesus do Itabapoana, Rio de Janeiro]>
CIDADE NOVA
<[OK] Arcgis - Geocode [Cidade Nova, Rio de Janeiro]>
ESTACIO
<[OK] Arcgis - Geocode [Estácio, Rio de Janeiro]>
GAMBOA
<[OK] Arcgis - Geocode [Gamboa, Rio de Janeiro]>
LAPA
<[OK] Arcgis - Geocode [Lapa, Rio de Janeiro]>
MANGUEIRA
<[OK] Arcgis - Geocode [Mangueira, Sapucaia, Rio de Janeiro]>
PAQUETA
<[OK] Arcgis - Geocode [Paquetá, Rio de Janeiro]>
RIO COMPRIDO
<[OK] Arcgis - Geocode [Rio Comprido, Rio de Janeiro]>
SANTA TERESA
<[OK] Arcgis - Geocode [Santa Teresa, Rio de Janeiro]>
SANTO CRISTO
<[OK] Arcgis - Geocode [Santo Cristo, Rio de Janeiro]>
SAUDE
<[OK] Arcgis - Geocode [Saúde, Rio de Janeiro]>
VASCO DA GAMA
<[OK] Arcgis - Geocode [Vasco da Gama, Rio de J

<[OK] Arcgis - Geocode [Cascadura, Rio de Janeiro]>
COELHO NETO
<[OK] Arcgis - Geocode [Coelho Neto, Rio de Janeiro]>
COLEGIO
<[OK] Arcgis - Geocode [Colégio, Rio de Janeiro]>
COMPLEXO DO ALEMAO
<[OK] Arcgis - Geocode [Complexo do Alemão, Rio de Janeiro]>
CORDOVIL
<[OK] Arcgis - Geocode [Cordovil, Rio de Janeiro]>
COSTA BARROS
<[OK] Arcgis - Geocode [Costa Barros, Rio de Janeiro]>
ENGENHEIRO LEAL
<[OK] Arcgis - Geocode [Engenheiro Leal, Rio de Janeiro]>
ENGENHO DA RAINHA
<[OK] Arcgis - Geocode [Engenho da Rainha, Rio de Janeiro]>
GUADALUPE
<[OK] Arcgis - Geocode [Guadalupe, Rio de Janeiro]>
HONORIO GURGEL
<[OK] Arcgis - Geocode [Honório Gurgel, Rio de Janeiro]>
INHAUMA
<[OK] Arcgis - Geocode [Inhaúma, Rio de Janeiro]>
IRAJA
<[OK] Arcgis - Geocode [Irajá, Rio de Janeiro]>
JARDIM AMERICA
<[OK] Arcgis - Geocode [Jardim América, Rio de Janeiro]>
MADUREIRA
<[OK] Arcgis - Geocode [Madureira, Rio de Janeiro]>
MARECHAL HERMES
<[OK] Arcgis - Geocode [Marechal Hermes, Rio de Janeiro]>
OSWALDO CR

In [16]:
df['LAT'] = lat

In [17]:
df['LON'] = long

In [18]:
# DROP ERROR JACARE / LARANJEIRAS / CENTRO  / MANGUEIRA -22.905917, -43.235340
df = df.drop([89,27,4,9])

In [19]:
df.shape

(157, 5)

In [20]:
# ERRO BAIRRO JACARÉ
jac = {
    'ZONA' : ['NORTE','SUL','CENTRAL','CENTRAL'],
    'SUB' : ['GRANDE MÉIER','ZONA SUL','CENTRO HISTÓRICO E ZONA PORTUÁRIA','CENTRO HISTÓRICO E ZONA PORTUÁRIA'],
    'BAIRRO' : ['JACARE','LARANJEIRAS','CENTRO','MANGUEIRA'],
    'LAT' : [float(-22.891399),float(-22.934578),float(-22.908076),float(-22.905917)],
    'LON' : [float(-43.258224),float(-43.189662),float(-43.182324),float(-43.235340)]
}

jac = pd.DataFrame(jac, index=[89,27,4,9])
df = df.append(jac , ignore_index=False, sort=False)

In [21]:
df

,BAIRRO,SUB,ZONA,LAT,LON
0,SAO CRISTOVAO,CENTRO HISTÓRICO E ZONA PORTUÁRIA,CENTRAL,-22.900200,-43.230240
1,BENFICA,CENTRO HISTÓRICO E ZONA PORTUÁRIA,CENTRAL,-22.889890,-43.245620
2,CAJU,CENTRO HISTÓRICO E ZONA PORTUÁRIA,CENTRAL,-22.879920,-43.222180
3,CATUMBI,CENTRO HISTÓRICO E ZONA PORTUÁRIA,CENTRAL,-22.917590,-43.197040
5,CIDADE NOVA,CENTRO HISTÓRICO E ZONA PORTUÁRIA,CENTRAL,-22.909860,-43.200080
...,...,...,...,...,...
160,VISTA ALEGRE,ZONA NORTE,NORTE,-22.829950,-43.317950
89,JACARE,GRANDE MÉIER,NORTE,-22.891399,-43.258224
27,LARANJEIRAS,ZONA SUL,SUL,-22.934578,-43.189662
4,CENTRO,CENTRO HISTÓRICO E ZONA PORTUÁRIA,CENTRAL,-22.908076,-43.182324


In [22]:
df = df.sort_index()
df = df.reset_index()

In [23]:
df = df.drop(['index'],axis=1)

In [24]:
df.head()

,BAIRRO,SUB,ZONA,LAT,LON
0,SAO CRISTOVAO,CENTRO HISTÓRICO E ZONA PORTUÁRIA,CENTRAL,-22.900200,-43.230240
1,BENFICA,CENTRO HISTÓRICO E ZONA PORTUÁRIA,CENTRAL,-22.889890,-43.245620
2,CAJU,CENTRO HISTÓRICO E ZONA PORTUÁRIA,CENTRAL,-22.879920,-43.222180
3,CATUMBI,CENTRO HISTÓRICO E ZONA PORTUÁRIA,CENTRAL,-22.917590,-43.197040
4,CENTRO,CENTRO HISTÓRICO E ZONA PORTUÁRIA,CENTRAL,-22.908076,-43.182324


In [25]:
df.shape

(161, 5)

## Creating Maps / Visualizations

In [26]:
import folium
import branca

In [27]:
g = geocoder.arcgis('RIO DE JANEIRO')
lat_lng_coords = g.latlng
latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are -22.91215999999997, -43.17500999999993.


# All Neighbourhoods map

In [28]:
# create map of New York using latitude and longitude values
map_rj = folium.Map(tiles='Stamen Terrain', location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, bairro, zona in zip(df['LAT'], df['LON'], df['BAIRRO'], df['ZONA']):
    label = '{}, {}'.format(bairro, zona)
    label = folium.Popup(label, parse_html=True)

    
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_rj)  
    
map_rj

# Regions map

In [32]:
# create map of New York using latitude and longitude values
map_rj = folium.Map(tiles='Stamen Terrain', location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, bairro, zona in zip(df['LAT'], df['LON'], df['BAIRRO'], df['ZONA']):
    label = '{}, {}'.format(bairro, zona)
    label = folium.Popup(label, parse_html=True)
    
    # Coloring Districs
    if zona == 'CENTRAL':
        c = 'blue'
    elif zona == 'NORTE':
        c = 'red'
    elif zona == 'SUL':
        c = 'orange'
    else:
        c = 'green'
        
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=c,
        fill=True,
        fill_color=c,
        fill_opacity=0.7,
        parse_html=False).add_to(map_rj)  
    
legend_html =   '''
                <div style="position: fixed; 
                            bottom: 10px; left: 10px; width: 190px; height: 45px; 
                            border:2px solid grey; z-index:9999; font-size:14px;
                            ">
                              &nbsp; CENTRAL > <i class="fa fa-circle" aria-hidden="true" style="color:blue"></i>
                              NORTE > <i class="fa fa-circle" aria-hidden="true" style="color:red"></i><br>
                              &nbsp; SUL > <i class="fa fa-circle" aria-hidden="true" style="color:orange"></i>
                              OESTE > <i class="fa fa-circle" aria-hidden="true" style="color:green"></i>

                </div>
                ''' 

map_rj.get_root().html.add_child(folium.Element(legend_html))
    
    
map_rj

# Subprefecture map

In [33]:
# create map of New York using latitude and longitude values
map_rj = folium.Map(tiles='Stamen Terrain' ,location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, bairro, zona in zip(df['LAT'], df['LON'], df['BAIRRO'], df['SUB']):
    label = '{}'.format(bairro)
    label = folium.Popup(label, parse_html=True)
    
    # Coloring Districs
    if zona == 'CENTRO HISTÓRICO E ZONA PORTUÁRIA':
        c = 'blue'
    elif zona == 'ZONA NORTE':
        c = 'yellow'
    elif zona == 'ILHA DO GOVERNADOR':
        c = 'grey'
    elif zona == 'GRANDE MÉIER':
        c = 'pink'
    elif zona == 'GRANDE TIJUCA':
        c = 'maroon'
    elif zona == 'ZONA OESTE':
        c = 'purple'
    elif zona == 'GRANDE BANGU':
        c = 'red'
    elif zona == 'BARRA DA TIJUCA E BAIXADA DE JACAREPAGUÁ':
        c = 'green'
    else:
        c = 'orange'
        
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=c,
        fill=True,
        fill_color=c,
        fill_opacity=0.7,
        parse_html=False).add_to(map_rj)  
    
legend_html =   '''
                <div style="position: fixed; 
                            bottom: 10px; left: 10px; width: 260px; height: 60px; 
                            border:2px solid grey; z-index:9999; font-size:14px;
                            ">
                              &nbsp; Imperial > <i class="fa fa-circle" aria-hidden="true" style="color:blue"></i>
                              Zona N > <i class="fa fa-circle" aria-hidden="true" style="color:yellow"></i>
                              Ilha > <i class="fa fa-circle" aria-hidden="true" style="color:grey"></i><br>
                              &nbsp; G Meier > <i class="fa fa-circle" aria-hidden="true" style="color:pink"></i>
                              G Tijuc > <i class="fa fa-circle" aria-hidden="true" style="color:brown"></i>
                              Zona O > <i class="fa fa-circle" aria-hidden="true" style="color:purple"></i><br>
                              &nbsp; G Bangu > <i class="fa fa-circle" aria-hidden="true" style="color:red"></i>
                              Barra T > <i class="fa fa-circle" aria-hidden="true" style="color:green"></i>
                              Zona S > <i class="fa fa-circle" aria-hidden="true" style="color:orange"></i>

                </div>
                ''' 

map_rj.get_root().html.add_child(folium.Element(legend_html))
    
    
map_rj

In [31]:
df.to_csv('bairros.csv')